<a href="https://colab.research.google.com/github/YunTaeHyeon/SmartBuildingBigDataAnalyze/blob/main/dataAnalyze/correlation_of_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import re
import csv
import gensim
from sklearn.preprocessing import normalize
from itertools import combinations

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 코사인 유사도 0.9 이상만

In [8]:
# CSV 파일 불러오기
data = pd.read_csv('/content/drive/MyDrive/경진대회 DATA/train_data.csv', encoding='cp1252')

# 열 선택
selected_columns = data[['temp', 'humi', 'co2', 'dust_pm1', 'dust_pm_25', 'dust_pm_10', 'illuminance', 'voc', 'eco2', 'PIR']]

# 결측치가 있는 행 제거
data_without_na = selected_columns.dropna()

# 데이터 정규화
normalized_data = normalize(data_without_na.values, axis=0)

# 코사인 유사도 계산
cosine_similarities = {}

for col1, col2 in combinations(data_without_na.columns, 2):
    vector1 = normalized_data[:, data_without_na.columns.get_loc(col1)]
    vector2 = normalized_data[:, data_without_na.columns.get_loc(col2)]
    cosine_similarity = np.dot(vector1, vector2) / (np.linalg.norm(vector1) * np.linalg.norm(vector2))
    cosine_similarities[(col1, col2)] = cosine_similarity

for (col1, col2), similarity in cosine_similarities.items():
    if similarity >= 0.9:
        print(f"코사인 유사도 {col1} , {col2}: {similarity}")

코사인 유사도 temp , humi: 0.9782043770661988
코사인 유사도 temp , co2: 0.913889930701971
코사인 유사도 humi , co2: 0.9043288176115872
코사인 유사도 dust_pm1 , dust_pm_25: 0.9961852546511202
코사인 유사도 dust_pm1 , dust_pm_10: 0.9682329690902972
코사인 유사도 dust_pm_25 , dust_pm_10: 0.9863037549889474


# PIR과 다른 모든 열들과의 유사도

In [14]:
# CSV 파일 불러오기
data_with_na = pd.read_csv('/content/drive/MyDrive/경진대회 DATA/train_data.csv', encoding='cp1252')

# 결측치가 있는 행 제거
data = data_with_na.dropna()

# 'PIR' 열 선택
pir_column = data['PIR']

# 다른 열들 선택
other_columns = data[['temp', 'humi', 'co2', 'dust_pm1', 'dust_pm_25', 'dust_pm_10', 'illuminance', 'voc', 'eco2']]

# 데이터 정규화
normalized_pir = normalize(pir_column.values.reshape(1, -1))
normalized_other = normalize(other_columns.values, axis=0)

# 코사인 유사도 계산
cosine_similarities = {}

for column_name, column_values in other_columns.items():
    vector = normalized_other[:, other_columns.columns.get_loc(column_name)]
    cosine_similarity = np.dot(normalized_pir, vector) / (np.linalg.norm(normalized_pir) * np.linalg.norm(vector))
    cosine_similarities[column_name] = cosine_similarity

# 결과 출력
for column_name, similarity in cosine_similarities.items():
    print(f"PIR과의 코사인 유사도 {column_name}: {similarity}") #쓰지 않아도 될것으로 예상

PIR과의 코사인 유사도 temp: [0.12694015]
PIR과의 코사인 유사도 humi: [0.12823687]
PIR과의 코사인 유사도 co2: [0.15763484]
PIR과의 코사인 유사도 dust_pm1: [0.09654377]
PIR과의 코사인 유사도 dust_pm_25: [0.0969877]
PIR과의 코사인 유사도 dust_pm_10: [0.09553409]
PIR과의 코사인 유사도 illuminance: [0.11315175]
PIR과의 코사인 유사도 voc: [0.03275457]
PIR과의 코사인 유사도 eco2: [0.13158344]
